# Creating Gradio App For Deployment of the Finetuned LLM model

Here I have written the Code to load the Python_Code Generator model fintuned from the Base LLama2 LLM Model from Huggingface Repo and then Deploying the Model using Gradio Libray in the Google Colab GPU.

The Model will run in the Goolge Colab backend GPU

In [ ]:
!pip install transformers accelerate gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 

**Importing the Dependencies**

In [ ]:
import os

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
os.environ["HF_TOKEN"] = "Hugging_face_Token_For_Reading_Persmission"

In [ ]:
model_name = "Riddhankur/llama_2_finetune"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [ ]:
def build_llama2_prompt(messages):
  start_prompt = '<s>[INST] '
  end_prompt = ' [/INST]'
  conversation=[]
  for index,message in enumerate(messages):
    if message["role"]=="system" and index==0:
      conversation.append(f"<<SYS>>\n{message['content']}\n<</SYS>>\n")
    elif message["role"]=="user":
      conversation.append(f"{message['content']}")
    else:
      conversation.append(f"{message['content']}</s>")
  prompt=start_prompt+"".join(conversation)+end_prompt
  return prompt


In [ ]:
messages=[{"role":"system","content":"Write a Pyhton code for Binary Search Algorithm"}]
instruction="Consider the array [2,6,8,1,3,9] and find the number8"
messages.append({"role":"user","content":instruction})
input_text=build_llama2_prompt(messages)
print(input_text)


<s>[INST] <<SYS>>
Write a Pyhton code for Binary Search Algorithm
<</SYS>>
Consider the array [2,6,8,1,3,9] and find the number8 [/INST]


In [ ]:
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

In [ ]:
input_ids

{'input_ids': tensor([[    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          6113,   263, 10772,   400,   265,   775,   363, 29479, 11856, 29068,
            13, 29966,   829, 14816, 29903,  6778,    13, 13696,  1241,   278,
          1409,   518, 29906, 29892, 29953, 29892, 29947, 29892, 29896, 29892,
         29941, 29892, 29929, 29962,   322,  1284,   278,  1353, 29947,   518,
         29914, 25580, 29962]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
response = model.generate(**input_ids, max_new_tokens=512).to("cuda")

In [ ]:
response

tensor([[    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          6113,   263, 10772,   400,   265,   775,   363, 29479, 11856, 29068,
            13, 29966,   829, 14816, 29903,  6778,    13, 13696,  1241,   278,
          1409,   518, 29906, 29892, 29953, 29892, 29947, 29892, 29896, 29892,
         29941, 29892, 29929, 29962,   322,  1284,   278,  1353, 29947,   518,
         29914, 25580, 29962, 29871, 18585, 29991,  2266,   338,   385,  1342,
           310,   920,   366,  1033,  2334,   263,  7581,  2740,  5687,   297,
          5132, 29901,    13, 28956,    13,  1753,  7581, 29918,  4478, 29898,
          2749, 29892,  3646,  1125,    13,  1678,  9995,    13,  1678,  2431,
          9514,   263,  7581,  2740,   373,   278,  1409,  3948, 29892,  3063,
           363,   278,  3646,   995, 29889,    13,  1678,  9995,    13,  1678,
          2175,   353, 29871, 29900,    13,  1678,  1492,   353,  7431, 29898,
          2749, 29897,   448, 29871, 29896,    13,  

In [ ]:
output = tokenizer.decode(response[0], skip_special_tokens=True)
print(output)

[INST] <<SYS>>
Write a Pyhton code for Binary Search Algorithm
<</SYS>>
Consider the array [2,6,8,1,3,9] and find the number8 [/INST]  Sure! Here is an example of how you could implement a binary search algorithm in Python:
```
def binary_search(arr, target):
    """
    Performs a binary search on the array arr, looking for the target value.
    """
    left = 0
    right = len(arr) - 1

    while left < right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1

    # If we reach this point, the target value was not found in the array
    return -1

# Test the binary search function
arr = [2, 6, 8, 1, 3, 9]
result = binary_search(arr, 8)
print(result)
```
This code defines a `binary_search` function that takes an array and a target value as input, and returns the index of the target value in the array if it is found, or -1 if it is not found.

The f

**Deploy LLM with Gradio as API**

In [ ]:
import json
import gradio as gr

In [ ]:
def get_response(input_data):
    input_data = json.loads(input_data)
    input_text = input_data["query"]
    input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
    response = model.generate(**input_ids, max_new_tokens=512).to("cuda")
    output = tokenizer.decode(response[0], skip_special_tokens=True)
    return {"output": output}

In [ ]:
demo = gr.Interface(
    fn=get_response,
    inputs="json",
    outputs="json"
)

demo.queue().launch(share=True, debug=True, show_error=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2497cf7c0672ec8ed7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2497cf7c0672ec8ed7.gradio.live
